In [18]:
import metapy
import pandas as pd
import numpy as np
from pyspark import SparkContext
# import boto3

In [19]:
metapy.log_to_stderr()

In [20]:
# We'll use a reduced version of the dataset while we configure our blob storage. The exercise does continue, albeit with fewer data to work on.
# data = pd.read_csv('../datasets/news_reduced.csv', encoding="utf-8")
data = pd.read_csv('../datasets/news.csv', encoding="utf-8")

In [21]:
data.head(20)

,id,id_news,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
5,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017.0,1.0,NaN,"LONDON — Queen Elizabeth II, who has been b..."
6,6,17289,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017.0,1.0,NaN,BEIJING — President Tsai of Taiwan sharpl...
7,7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017.0,2.0,NaN,"Danny Cahill stood, slightly dazed, in a blizz..."
8,8,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016.0,12.0,NaN,"Just how is Hillary Kerr, the founder of ..."
9,9,17292,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016.0,12.0,NaN,Angels are everywhere in the Muñiz family’s ap...


In [22]:
news = data['title'] + ' ' + data['content']
metadata = data[['id_news','publication','author','date','year', 'month']]
metadata.head(10)

,id_news,publication,author,date,year,month
0,17283,New York Times,Carl Hulse,2016-12-31,2016.0,12.0
1,17284,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0
2,17285,New York Times,Margalit Fox,2017-01-06,2017.0,1.0
3,17286,New York Times,William McDonald,2017-04-10,2017.0,4.0
4,17287,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0
5,17288,New York Times,Sewell Chan,2017-01-02,2017.0,1.0
6,17289,New York Times,Javier C. Hernández,2017-01-02,2017.0,1.0
7,17290,New York Times,Gina Kolata,2017-02-08,2017.0,2.0
8,17291,New York Times,Katherine Rosman,2016-12-31,2016.0,12.0
9,17292,New York Times,Andy Newman,2016-12-31,2016.0,12.0


In [23]:
news_dat = pd.DataFrame(columns=['document'], data=news)
news_dat.head()

,document
0,House Republicans Fret About Winning Their Hea...
1,Rift Between Officers and Residents as Killing...
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi..."
4,Kim Jong-un Says North Korea Is Preparing to T...


In [24]:
news_dat.to_csv('../data/news/news.dat', header=None, index=None)
# news_dat.to_csv('../data/news_reduced/news_reduced.dat', header=None, index=None)

In [26]:
idx = metapy.index.make_inverted_index('../config/config.toml')

 > Counting lines in file: [=================================] 100% ETA 00:00:00                            
 
1561160163: [info]     Creating index: news/inv (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:119)
1561160163: [info]     Creating index: news/inv (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:119)
 > Tokenizing Docs: [============>                           ]  30% ETA 00:00:22                    
 
1561160173: [warning]  Empty document (id = 44242) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160173: [warning]  Empty document (id = 44242) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
 
 
1561160173: [warning]  Empty document (id = 44243) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta

1561160176: [warning]  Empty document (id = 53819) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160176: [warning]  Empty document (id = 53819) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
 
 
1561160176: [warning]  Empty document (id = 53822) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160176: [warning]  Empty document (id = 53822) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
 
 
1561160176: [warning]  Empty document (id = 53816) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160176: [warning]  Empty document (id = 53816) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/in

1561160176: [warning]  Empty document (id = 53933) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160176: [warning]  Empty document (id = 53933) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
 
 
1561160176: [warning]  Empty document (id = 53936) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160176: [warning]  Empty document (id = 53936) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
 
 
1561160176: [warning]  Empty document (id = 53942) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160176: [warning]  Empty document (id = 53942) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/in

 > Tokenizing Docs: [===============>                        ]  39% ETA 00:00:21 
 
1561160178: [warning]  Empty document (id = 57362) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160178: [warning]  Empty document (id = 57362) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
 
 
1561160178: [warning]  Empty document (id = 57383) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
1561160178: [warning]  Empty document (id = 57383) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/inverted_index.cpp:228)
 > Tokenizing Docs: [================>                       ]  41% ETA 00:00:20  
 
1561160178: [warning]  Empty document (id = 61890) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/invert

In [27]:
idx.num_docs()

142797

In [28]:
idx.unique_terms()

327764

In [29]:
idx.avg_doc_length()

446.1145935058594

In [30]:
idx.total_corpus_terms()

63703822

In [31]:
query = metapy.index.Document()
query.content('Airbus Subsidies') # query from AP news

In [32]:
ranker = metapy.index.OkapiBM25()

In [33]:
top_docs = ranker.score(idx, query, num_results=5)
top_docs

[(116978, 17.60701560974121),
 (44887, 16.50486946105957),
 (119522, 16.224567413330078),
 (118943, 16.13990592956543),
 (43530, 15.877269744873047)]

In [34]:
fidx = metapy.index.make_forward_index('../config/config.toml')

 > Counting lines in file: [=================================] 100% ETA 00:00:00                            
 
1561160219: [info]     Creating forward index: news/fwd (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:239)
1561160219: [info]     Creating forward index: news/fwd (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:239)
 > Tokenizing Docs: [============>                           ]  30% ETA 00:00:22                     
 
1561160229: [warning]  Empty document (id = 44242) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 > Tokenizing Docs: [============>                           ]  30% ETA 00:00:221561160229: [warning]  Empty document (id = 44242) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 
 
1561160229: [warning]  Empty document (id

 
 
1561160232: [warning]  Empty document (id = 53786) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160232: [warning]  Empty document (id = 53786) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 
 
1561160232: [warning]  Empty document (id = 53792) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160232: [warning]  Empty document (id = 53792) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 
 
1561160232: [warning]  Empty document (id = 53795) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160232: [warning]  Empty document (id = 53795) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/for

1561160233: [warning]  Empty document (id = 53915) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160233: [warning]  Empty document (id = 53915) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 
 
1561160233: [warning]  Empty document (id = 53918) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160233: [warning]  Empty document (id = 53918) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 
 
1561160233: [warning]  Empty document (id = 53924) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160233: [warning]  Empty document (id = 53924) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward

 > Tokenizing Docs: [===============>                        ]  39% ETA 00:00:22  
 
1561160234: [warning]  Empty document (id = 57362) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160234: [warning]  Empty document (id = 57362) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 
 
1561160234: [warning]  Empty document (id = 57383) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
1561160234: [warning]  Empty document (id = 57383) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_index.cpp:335)
 > Tokenizing Docs: [=================>                      ]  43% ETA 00:00:20  
 
1561160235: [warning]  Empty document (id = 61890) generated! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/index/forward_i

In [35]:
dset = metapy.learn.Dataset(fidx)

 > Loading instances into memory: [==========================] 100% ETA 00:00:00    
 


In [45]:
lda_inf = metapy.topics.LDACollapsedVB(dset, num_topics=4, alpha=0.1, beta=0.1)

In [46]:
lda_inf.run(num_iters=1000)

Initialization: [============================================] 100% ETA 00:00:00                      
Iteration 1 maximum change in gamma: 1.63971                                          
Iteration 2 maximum change in gamma: 0.486095                                         
Iteration 3 maximum change in gamma: 0.30679                                          
Iteration 4 maximum change in gamma: 0.429276                                         
Iteration 5 maximum change in gamma: 0.468586                                         
Iteration 6 maximum change in gamma: 0.588577                                         
Iteration 7 maximum change in gamma: 0.711016                                         
Iteration 8 maximum change in gamma: 0.931093                                         
Iteration 9 maximum change in gamma: 0.937031                                         
Iteration 10 maximum change in gamma: 1.12597                                         
Iteration 11 maximum change

Iteration 94 maximum change in gamma: 0.435625                                       
Iteration 95 maximum change in gamma: 0.434289                                        
Iteration 96 maximum change in gamma: 0.432025                                        
Iteration 97 maximum change in gamma: 0.316956                                        
Iteration 98 maximum change in gamma: 0.335342                                        
Iteration 99 maximum change in gamma: 0.338352                                        
Iteration 100 maximum change in gamma: 0.285441                                       
Iteration 101 maximum change in gamma: 0.207152                                       
Iteration 102 maximum change in gamma: 0.239577                                       
Iteration 103 maximum change in gamma: 0.266459                                       
Iteration 104 maximum change in gamma: 0.266181                                       
Iteration 105 maximum change in gamma: 0.233

Iteration 282 maximum change in gamma: 0.00835141                                    
Iteration 283 maximum change in gamma: 0.00850465                                     
Iteration 284 maximum change in gamma: 0.00865839                                     
Iteration 285 maximum change in gamma: 0.0088122                                      
Iteration 286 maximum change in gamma: 0.00896557                                     
Iteration 287 maximum change in gamma: 0.00911799                                     
Iteration 288 maximum change in gamma: 0.00926886                                     
Iteration 289 maximum change in gamma: 0.00941756                                     
Iteration 290 maximum change in gamma: 0.00956343                                     
Iteration 291 maximum change in gamma: 0.00970574                                     
Iteration 292 maximum change in gamma: 0.00984372                                     
Iteration 293 maximum change in gamma: 0.009

Iteration 470 maximum change in gamma: 0.0035075                                     
Iteration 471 maximum change in gamma: 0.00352537                                     
Iteration 472 maximum change in gamma: 0.00354271                                     
Iteration 473 maximum change in gamma: 0.00355952                                     
Iteration 474 maximum change in gamma: 0.00357578                                     
Iteration 475 maximum change in gamma: 0.00359147                                     
Iteration 476 maximum change in gamma: 0.00360657                                     
Iteration 477 maximum change in gamma: 0.00362108                                     
Iteration 478 maximum change in gamma: 0.00363497                                     
Iteration 479 maximum change in gamma: 0.00364825                                     
Iteration 480 maximum change in gamma: 0.00366088                                     
Iteration 481 maximum change in gamma: 0.003

Iteration 658 maximum change in gamma: 0.00101486                                    
Iteration 659 maximum change in gamma: 0.00100371                                     
Iteration 660 maximum change in gamma: 0.00099269                                     
1559618024: [info]     Finished maximum iterations, or found convergence! (C:/Users/appveyor/AppData/Local/Temp/1/pip-req-build-zhxsvdb2/deps/meta/src/topics/lda_cvb.cpp:60)


In [47]:
lda_inf.save('lda-cvb0')

In [48]:
model = metapy.topics.TopicModel('lda-cvb0')

 > Loading topic term probabilities: [=======================] 100% ETA 00:00:00 
 > Loading document topic probabilities: [===================] 100% ETA 00:00:00 


In [49]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0)]

[('</s>', 0.08251002753031933),
 ('<s>', 0.08251002487928022),
 ('new', 0.005312014112903867),
 ('time', 0.0033949843315053004),
 ('york', 0.003174583324484505),
 ('itâ', 0.0031529298944495392),
 ('œi', 0.0028695982055421507),
 ('times', 0.002774610632976567),
 ('people', 0.0026501197971943204),
 ('two', 0.002398750157799485)]

In [50]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1)]

[('<s>', 0.06764454226302818),
 ('</s>', 0.06764453762694526),
 ('police', 0.005480852180953524),
 ('united', 0.005403856100368241),
 ('people', 0.0044869427366391774),
 ('new', 0.0040253697214384975),
 ('state', 0.0040004758674713385),
 ('states', 0.0038056333857363783),
 ('government', 0.0035714223566911695),
 ('officials', 0.003468794288530383)]

In [51]:
scorer = metapy.topics.BLTermScorer(model)

In [52]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0, scorer=scorer)]

[('iâ', 0.006920094548716393),
 ('</s>', 0.006859510941861657),
 ('<s>', 0.006859506023880968),
 ('itâ', 0.004756069222011966),
 ('game', 0.004495549572285373),
 ('music', 0.0043051898055789795),
 ('season', 0.004176658204940037),
 ('art', 0.003965855989342987),
 ('film', 0.003536461435793073),
 ('youâ', 0.0035174874067713247)]

In [53]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1, scorer=scorer)]

[('police', 0.02745055692676468),
 ('islamic', 0.018068924937522518),
 ('military', 0.014735841071415123),
 ('syria', 0.013032721717181728),
 ('officers', 0.010663397046224876),
 ('government', 0.010650134984107607),
 ('officials', 0.010255331279227786),
 ('syrian', 0.008526051833094017),
 ('killed', 0.00830637778307719),
 ('united', 0.008202418699712777)]

In [54]:
model.topic_distribution(0)

<metapy.stats.Multinomial {0: 0.000448, 1: 0.000495, 2: 0.796254, 3: 0.202804}>

In [55]:
model.topic_distribution(900)

<metapy.stats.Multinomial {0: 0.000525, 1: 0.000416, 2: 0.249007, 3: 0.750052}>

In [ ]:
for num, (d_id, _) in enumerate(top_docs):
    content = idx.metadata(d_id).get('document')
    print("{}. {}...\n".format(num + 1, content))